# A short example

This notebook is a short example on how to use the `tof` package for making time-of-light diagrams of neutrons passing through a chopper cascade.

In [ ]:
import scipp as sc
import tof

Hz = sc.Unit('Hz')
deg = sc.Unit('deg')
meter = sc.Unit('m')

## Create a source pulse

We first create a source with one pulse containing 1 million neutrons whose distribution follows the ESS time and wavelength profiles (both thermal and cold neutrons are included).

In [ ]:
source = tof.Source(facility='ess', neutrons=1_000_000)
source

In [ ]:
source.plot()

## Chopper set-up

We create a list of choppers that will be included in our beamline.
In our case, we make two WFM choppers, and two frame-overlap choppers.
All choppers have 6 openings.

In [ ]:
choppers = [
    tof.Chopper(
        frequency=70.0 * Hz,
        open=sc.array(
            dims=['cutout'],
            values=[98.71, 155.49, 208.26, 257.32, 302.91, 345.3],
            unit='deg',
        ),
        close=sc.array(
            dims=['cutout'],
            values=[109.7, 170.79, 227.56, 280.33, 329.37, 375.0],
            unit='deg',
        ),
        phase=47.10 * deg,
        distance=6.6 * meter,
        name="WFM1",
    ),
    tof.Chopper(
        frequency=70 * Hz,
        open=sc.array(
            dims=['cutout'],
            values=[80.04, 141.1, 197.88, 250.67, 299.73, 345.0],
            unit='deg',
        ),
        close=sc.array(
            dims=['cutout'],
            values=[91.03, 156.4, 217.18, 269.97, 322.74, 375.0],
            unit='deg',
        ),
        phase=76.76 * deg,
        distance=7.1 * meter,
        name="WFM2",
    ),
    tof.Chopper(
        frequency=56 * Hz,
        open=sc.array(
            dims=['cutout'],
            values=[74.6, 139.6, 194.3, 245.3, 294.8, 347.2],
            unit='deg',
        ),
        close=sc.array(
            dims=['cutout'],
            values=[95.2, 162.8, 216.1, 263.1, 310.5, 371.6],
            unit='deg',
        ),
        phase=62.40 * deg,
        distance=8.8 * meter,
        name="Frame-overlap 1",
    ),
    tof.Chopper(
        frequency=28 * Hz,
        open=sc.array(
            dims=['cutout'],
            values=[98.0, 154.0, 206.8, 254.0, 299.0, 344.65],
            unit='deg',
        ),
        close=sc.array(
            dims=['cutout'],
            values=[134.6, 190.06, 237.01, 280.88, 323.56, 373.76],
            unit='deg',
        ),
        phase=12.27 * deg,
        distance=15.9 * meter,
        name="Frame-overlap 2",
    ),
    tof.Chopper(
        frequency=7 * Hz,
        open=sc.array(
            dims=['cutout'],
            values=[30.0],
            unit='deg',
        ),
        close=sc.array(
            dims=['cutout'],
            values=[140.0],
            unit='deg',
        ),
        phase=0 * deg,
        distance=22 * meter,
        name="Pulse-overlap",
    ),
]

## Detector set-up

We add a monitor 26 meters from the source, and a main detector 32 meters from the source.

In [ ]:
detectors = [
    tof.Detector(distance=26.0 * meter, name='monitor'),
    tof.Detector(distance=32.0 * meter, name='detector'),
]

## Run the model

We combine the `source`, `choppers`, and `detectors` into our `model`,
and then use the `.run()` method to execute the ray-tracing simulation.

In [ ]:
model = tof.Model(source=source, choppers=choppers, detectors=detectors)
model

In [ ]:
res = model.run()
res

## Results

### Plotting

We can plot the models as a whole (which will show the ray paths through the system),
and the individual components (which will show the counts each component is seeing).

In [ ]:
res.plot(max_rays=5000)

In [ ]:
res.detectors['monitor'].tofs.plot()

In [ ]:
res.choppers["Frame-overlap 2"].tofs.plot()

### Data inspection

Each component entry in the results objects holds all the information about the neutrons that reached that component.
The `.data` property of the object returns a data array, which has one pulse of neutrons.

In [ ]:
res.choppers['WFM1'].data

The `.tofs`, `.wavelengths`, `.birth_times`, and `.speeds` properties of the beamline components return a proxy object,
which gives access to the data they hold.

Accessing their `.data` returns a data group with both visible and blocked neutrons:

In [ ]:
res.choppers['WFM1'].tofs.data

To inspect individual components, one can for example use the `.data` property of the `.visible` accessor

In [ ]:
res.choppers['WFM1'].tofs.visible.data

This is a data group that contains a single pulse.

As these are Scipp data structures, they can be manipulated and plotted directly.
We first select the first (and only) pulse in the wavelength data using the `[0]` syntax,
and subsequently histogram and plot the data:

In [ ]:
res.choppers['WFM1'].wavelengths[0].data.hist(wavelength=500).plot()